
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Reading from a Streaming Query

## Objectives
1. Build a streaming DataFrame
1. Display streaming query results
1. Write streaming query results to table
1. Monitor the streaming query


### Classes referenced
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.ss/api/pyspark.sql.streaming.DataStreamReader.html" target="_blank">DataStreamReader</a>
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.ss/api/pyspark.sql.streaming.DataStreamWriter.html" target="_blank">DataStreamWriter</a>
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.ss/api/pyspark.sql.streaming.StreamingQuery.html" target="_blank">StreamingQuery</a>

In [0]:
%run ./Includes/Classroom-Setup-01.1

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


Resetting the learning environment (structured_streaming):
| No action taken

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/data-engineer-learning-path/v04"

Validating the locally installed datasets:
| listing local files...(8 seconds)
| validation completed...(9 seconds total)

Creating & using the schema "labuser7987656_1729114308_8qsa_da_sdlt_structured_streaming" in the catalog "hive_metastore"...(6 seconds)

Predefined tables in "labuser7987656_1729114308_8qsa_da_sdlt_structured_streaming":
| -none-

Predefined paths variables:
| DA.paths.working_dir: dbfs:/mnt/dbacademy-users/labuser7987656_1729114308@vocareum.com/databricks-streaming-and-delta-live-tables/structured_streaming
| DA.paths.user_db:     dbfs:/mnt/dbacademy-users/labuser7987656_1729114308@vocareum.com/databricks-streaming-and-delta-live-tables/structured_streaming/database.db
| DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/data-engineer-learning-path/v04
| DA.paths.checkpoints: dbfs:/mn

## Prototyping in Batch Mode

Explore the dataset and test out transformation logic using batch dataframes.

In [0]:
from pyspark.sql.functions import col, approx_count_distinct, count

batch_df = (spark.read
              .load(DA.paths.events)
              .filter(col("traffic_source") == "email")
              .withColumn("mobile", col("device").isin(["iOS", "Android"]))
              .select("user_id", "event_timestamp", "mobile")
           )

print(batch_df.isStreaming)

display(batch_df)

False


user_id,event_timestamp,mobile
UA000000106527304,1593733998028702,false
UA000000106506669,1593817553389549,true
UA000000106515529,1593809961957359,false
UA000000106530934,1593617743761830,false
UA000000106520932,1593805531448682,false
UA000000106484593,1593604165030991,true
UA000000106509125,1593610748505664,true
UA000000106479002,1593848889855972,false
UA000000106535488,1593615934041502,true
UA000000106500611,1593816433585044,false



## Build streaming DataFrames

Switching from batch to stream is easy! 

Change the code from `spark.read` to `spark.readStream` - everything else, including the transformation logic remain unchanged.

In [0]:
from pyspark.sql.functions import col, approx_count_distinct, count
 
streaming_df = (spark.readStream.load(DA.paths.events)
                      .filter(col("traffic_source") == "email")
                      .withColumn("mobile", col("device").isin(["iOS", "Android"]))
                      .select("user_id", "event_timestamp", "mobile")
                   )

print(streaming_df.isStreaming)

display(streaming_df, streamName = "display_user_devices")

user_id,event_timestamp,mobile
UA000000106527304,1593733998028702,false
UA000000106506669,1593817553389549,true
UA000000106515529,1593809961957359,false
UA000000106530934,1593617743761830,false
UA000000106520932,1593805531448682,false
UA000000106484593,1593604165030991,true
UA000000106509125,1593610748505664,true
UA000000106479002,1593848889855972,false
UA000000106535488,1593615934041502,true
UA000000106500611,1593816433585044,false



### Write streaming query results

Take the final streaming DataFrame (our result table) and write it to a Delta Table sink in `append` mode. In this labs setup, the table will be created in the `hive_metastore`. In the real world, we strongly recommend all new Delta tables to be created and managed in Unity Catalog.

**NOTE:** `append` mode is the default mode when writing stateless queries to sink.

In [0]:
checkpoint_path = f"{DA.paths.working_dir}/email_traffic"

devices_query = (streaming_df
                  .writeStream
                  .outputMode("append")
                  .format("delta")
                  .queryName("email_traffic")
                  .trigger(processingTime="1 second")
                  .option("checkpointLocation", checkpoint_path)
                  .toTable("email_traffic_only")
                )

In [0]:
DA.block_until_stream_is_ready(devices_query)

Processed 69 of 2 batches...
The stream is now active with 69 batches having been processed.


### Monitor streaming query

Use the streaming query handle to monitor and control it.

In [0]:
devices_query.name

'email_traffic'


Let's see the query status.

In [0]:
devices_query.status

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}


[lastProgress](https://spark.apache.org/docs/3.5.2/structured-streaming-programming-guide.html#managing-streaming-queries) gives us metrics from the previous query

In [0]:
devices_query.lastProgress

{'id': 'f13140dd-9ce0-4394-ac3d-11d39c4bad70',
 'runId': 'b60efe37-ce1d-4238-a4cc-9e8471107c7b',
 'name': 'email_traffic',
 'timestamp': '2024-10-16T22:06:10.001Z',
 'batchId': 1,
 'batchDuration': 70,
 'numInputRows': 0,
 'inputRowsPerSecond': 0.0,
 'processedRowsPerSecond': 0.0,
 'durationMs': {'latestOffset': 70, 'triggerExecution': 70},
 'stateOperators': [],
 'sources': [{'description': 'DeltaSource[dbfs:/mnt/dbacademy-datasets/data-engineer-learning-path/v04/ecommerce/delta/events_hist]',
   'startOffset': {'sourceVersion': 1,
    'reservoirId': '0acdb449-9b2f-443e-998c-2685de28d985',
    'reservoirVersion': 1,
    'index': -1,
    'isStartingVersion': False},
   'endOffset': {'sourceVersion': 1,
    'reservoirId': '0acdb449-9b2f-443e-998c-2685de28d985',
    'reservoirVersion': 1,
    'index': -1,
    'isStartingVersion': False},
   'latestOffset': None,
   'numInputRows': 0,
   'inputRowsPerSecond': 0.0,
   'processedRowsPerSecond': 0.0,
   'metrics': {'numBytesOutstanding': '0'

In [0]:
import time
# Run for 10 more seconds
time.sleep(10) 

devices_query.stop()

In [0]:
devices_query.awaitTermination()


[awaitTermination](https://spark.apache.org/docs/3.5.2/structured-streaming-programming-guide.html#managing-streaming-queries) blocks the current thread until the streaming query is terminated.

For stand-alone structured streaming applications, this is used to prevent the main thread from terminating while the streaming query is still executing. In the context of this training environment, it's useful in case you use **Run all** to run the notebook. This prevents subsequent command cells from executing until the streaming query has fully terminated.


## Classroom Cleanup

Run the cell below to clean up resources.

In [0]:
DA.cleanup()


&copy; 2024 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>